In [378]:
from xml.dom import minidom 
import random
import time

In [492]:
def read_in(xml_file):
    
    doc = minidom.parse(xml_file)  ### takes in the xml file 

    variables_length = len(doc.getElementsByTagName('VARIABLE')) 
    definitions_length = len(doc.getElementsByTagName('DEFINITION'))

    variable_dict = {}

    for i in range(variables_length):
        item = doc.getElementsByTagName('VARIABLE')[i].getElementsByTagName('NAME') ### extracts the name nodes 
        outcomes = doc.getElementsByTagName('VARIABLE')[i].getElementsByTagName('OUTCOME') ### extracts the outcomes nodes  

        sub_list = []
        for j in outcomes:
            sub_list.append(j.toxml()[9:-10]) ### looks for the outcomes 


        for j in item:
            variable_dict[j.toxml()[6:-7]] = sub_list ### assigns outcomes to variables 


    definitions = doc.getElementsByTagName('DEFINITION') ### looks for the definitions nodes

    test_list = []
    for i in definitions: ### for each definition we look for what it's for, what is given and the table of probabilities
        fors = i.getElementsByTagName('FOR')
        givens = i.getElementsByTagName('GIVEN')
        items = i.getElementsByTagName('TABLE')

        fors_list = []
        for j in fors:
            fors_list.append(j.toxml()[5:-6]) ### finds variables defined by probabilities

        givens_list = []
        for j in givens:
            givens_list.append(j.toxml()[7:-8]) ### finds what we are given/parents of the variables

        prob_list = []
        for j in items:
            s = j.toxml()[7:-8].split(' ') ### splits all the values contained in the table
            #print(s)
            while '' in s: 
                s.remove('') ### removes all '' from the list

            sub_list = []
            for k in s:
                
                if k[0].isdigit(): ### looks for all values that are a probility (if the value is a number)

                    if k[-1] == 't':
                        prob_list.append(float(k[:-4])) ### there is /n/t at the end of each value that is at the end of a row
                        ### if this case extract the probability
                    else:
                        prob_list.append(float(k))
                        ### if another case we assume it's a probability
                        
            #prob_list.append(sub_list)

        fors_list.append(givens_list)
        fors_list.append(prob_list)
        test_list.append(fors_list) ### adds all values that we extracted from this loop 

    ret_dict = {}
    
    for i in range(variables_length): ### creates our data structure that resembles a graph/tree
        test_list[i].insert(1, variable_dict[test_list[i][0]])
        ret_dict[test_list[i][0]] = test_list[i][1:]
    
    ret_dict = create_probability_table(ret_dict) ### separates the rows of each table
    
    ret_dict = organize_data(ret_dict)
    
    return ret_dict

In [164]:
def create_probability_table(sub_dict): ### separates rows of the list of probabilites
    
    for key in sub_dict:
        
        sub_list = []
        
        if sub_dict[key][1] != []: ### if the value has parents 
        
            columns = len(sub_dict[key][0]) ### number of entries is based on the number of outcomes a value has 
            
            for i in range(0, len(sub_dict[key][2]), columns): ### the rows has the same number of values as the columns 
                sub_list.append(sub_dict[key][2][i:(i+columns)])
                
            sub_dict[key][2] = sub_list ### the table is updated
        
        else:
            continue
            
    return sub_dict

In [489]:
def organize_data(data):
    
    sub_list = []
    
    sub_dict = {}
    
    length = len(list(data.keys()))
    
    for key in data:
        if data[key][1] == []:
            sub_list.insert(0, key)
    
    #while len(sub_list) != length:
    for parent in sub_list:
        for key in data: 
            if parent in data[key][1]:
                sub_list.append(key)
            
        
    for i in sub_list:
        
        sub_dict[i] = data[i]
                    
    
    return sub_dict

In [ ]:
'''
Here is an example of the data structure

{
 'B': [['true', 'false'], [], [0.001, 0.999]],
 'E': [['true', 'false'], [], [0.002, 0.998]],
 'A': [['true', 'false'], ['B', 'E'], [[0.95, 0.05], [0.94, 0.06], [0.29, 0.71], [0.001, 0.999]]],
 'J': [['true', 'false'], ['A'], [[0.9, 0.1], [0.05, 0.95]]],
 'M': [['true', 'false'], ['A'], [[0.7, 0.3], [0.01, 0.99]]]
}
 '''

In [ ]:
'''
function ENUMERATION-ASK(X,e,bn) returns a distribution over X 

inputs: X, the queryvariable 
e, observedvalues forvariables E 
bn, a Bayes net with variables {X}∪E ∪ Y /* Y = hidden variables */ 

Q(X)←a distribution over X, initially empty 

for each value x_i of X do 
    Q(xi)←ENUMERATE-ALL(bn.VARS,exi) #where e_xi is e extended with X = x_i 

return NORMALIZE(Q(X))
'''

In [ ]:
'''
function ENUMERATE-ALL(vars,e) returns a real number 
if EMPTY?(vars) then return 1.0 
Y ←FIRST(vars) 
if Y has value y in e 
    then return P(y | parents(Y )) × ENUMERATE-ALL(REST(vars),e) 
else return SUM_y P(y | parents(Y )) × ENUMERATE-ALL(REST(vars),ey) 
    #where ey is e extendedwith Y = y
'''

# Exact Inference

In [494]:
class Enumeration:
    
    def __init__(self, X, e, data): ### sets all initials
        
        self.query = X ### string query
        
        self.data = data ### possibly turn into graph/tree
        
        self.query_outcomes = self.data[self.query][0] ### finds all of the outcomes for the query
        
        self.evidence_dict = {} ### evidence
        
        e = e.split(' ') ### assume e is a string of variables followed by their values
        
        for i in range(0, len(e), 2): ### assume evidence values are of the structure J true 
            self.evidence_dict[e[i]] = e[i + 1]   
            
        self.variables = list(self.data.keys())  
        #print(self.data)

        self.enumeration_ask() ### automatically starts the process
        
    def enumeration_ask(self):
       
        Q = {}
        for outcome in self.query_outcomes: 
            
            self.evidence_dict[self.query] = outcome ### extends evidence with the outcomes 
            Q[outcome] = self.enumerate_all(self.variables) ### enumerates each outcome
        
        print(Q)
        
        print(self.normalize(Q))
        
        #return self.normalize(Q)
    
    def enumerate_all(self, varz):

        if varz == []:
            return 1

        y = varz[0]
        
        #print(self.variables)
        
        #print(y)

        if y in self.evidence_dict: ### recursively finds all probabilties of variables

            return self.find_probability(y) * self.enumerate_all(varz[1:])

        else:
            total = 0
            
            current_outcomes = self.data[y][0]
            
            #index = 0
            for i in current_outcomes: ### changes the value of the current variable 
                
                self.evidence_dict[y] = i
                total += self.find_probability(y) * self.enumerate_all(varz[1:])
                #index += 1
                
            del self.evidence_dict[y]
            
            return total
        
        
    def find_probability(self, occurance): ### searches the probability
        
        column = 0
        for outcome in self.data[occurance][0]: ### based on the occurance we can find what row we are in 
            
            if outcome == self.evidence_dict[occurance]:
                break
            
            column += 1
        
        if self.data[occurance][1] == []: ### if there are no parents, it is simply a list and we extract the value using the column
            
            return self.data[occurance][2][column]
        
        else:
            
            row = self.search_table(occurance, self.data[occurance][2]) ### find the row based on the current occurance 
            
            return row[column] ### return value 
            
        
    def normalize(self, sub_dict): 
        
        total = 0
        for i in sub_dict:
            total += sub_dict[i]
            
        for i in sub_dict:
            sub_dict[i] /= total
            
        return sub_dict   
    
    
    def search_table(self, occurance, table): 
        
        indicies = []

        for parent in self.data[occurance][1]: ### using the parents we can find our probability using a list of indicies

            index = 0
            for i in self.data[parent][0]: ### finds the index based on the current evidence 

                if i == self.evidence_dict[parent]:
                    indicies.append(index)
                    break
                    
                index += 1

        if len(indicies) == 1:
            return self.data[occurance][2][indicies[0]] ### if there is only 1 parent just return the row based on the found index 
        
        else:
            
            fodder = table.copy()
            
            for index in indicies[:-1]: ### go through each index and reduce table based on each index and respective variable

                x = int(len(table)/len(self.data[occurance][0]))

                fodder = fodder[x*index : x*index + x] ### breaks down search 
            
            return fodder[indicies[-1]] ### conclude by using last index in list

In [238]:
def mybninferencer(xml_file, X, e):
    
    data = read_in(xml_file)
    ret = Enumeration(X, e, data)

In [495]:
xml_file = 'dog-problem.xml'
X = 'dog-out'
e = 'light-on true'

start = time.time()
mybninferencer(xml_file, X, e)
end = time.time()


{'true': 0.09411575, 'false': 0.038384249999999995}
{'true': 0.7103075471698113, 'false': 0.28969245283018863}


In [ ]:
'''
function LIKELIHOOD-WEIGHTING(X,e,bn,N) returns an estimate of P(X|e) 
    inputs: X, the query variable 
    e, observed values forvariables E 
    bn, a Bayesian network specifyingjoint distribution P(X1,...,Xn) 
    N, the total number of samples to be generated 
    
    local variables: W, a vectorof weighted counts for each value of X, initially zero 
    
    for j = 1 toN do 
    
        x,w ←WEIGHTED-SAMPLE(bn,e) 
        W[x]←W[x]+w where x is the value of X in x 
        
    return NORMALIZE(W)


function WEIGHTED-SAMPLE(bn,e) returns an event and a weight 
    w ←1; x←an event with n elements initialized from e 
    
    foreach variable Xi in X1,...,Xn do 
        if Xi is an evidence variable with value xi in e 
            then w ←w × P(Xi = xi | parents(Xi)) 
            
        else x[i]←a randomsample from P(Xi | parents(Xi)) 
        
    return x, w

'''

In [ ]:
'''
function GIBBS-ASK(X,e,bn,N) returns an estimate of P(X|e) 
    local variables: N, a vector of counts for each value of X, initially zero 
    Z, the nonevidence variables in bn 
    x, the current state of the network, initially copied from e 

    initialize x with randomvalues for the variables in Z 
    for j = 1 to N do 
        for each Zi in Z do 
            set the value of Zi in x by sampling from P(Zi|mb(Zi))
                N[x]←N[x]+1where x is the value of X in x 
    return NORMALIZE(N)
'''

In [ ]:
'''
function PRIOR-SAMPLE(bn) returns an event sampled from the prior speciﬁed by bn 
    inputs: bn, a Bayesian network specifyingjoint distribution P(X1,...,Xn) 
    
    x←an event with n elements 
    
    foreach variable Xi in X1,...,Xn do 
    
        x[i]←a random sample from P(Xi | parents(Xi)) 
        
    return x

function REJECTION-SAMPLING(X,e,bn,N) returns an estimate of P(X|e)
inputs: X, the query variable 
    e, observed values for variables E 
    bn, a Bayesian network 
    N, the total numberof samples to be generated 
    local variables: N, a vector of counts for each value of X, initially zero 
    
    for j = 1 to N do 
        x←PRIOR-SAMPLE(bn) 
        if x is consistent with e 
            N[x]←N[x]+1 where x is the value of X in x 
        
    return NORMALIZE(N)
'''

# Approximate Inference

In [374]:
def mybn_approximate_inferencer(number_samples, xml_file, X, e):
    
    data = read_in(xml_file)
    ret = Rejection_sampling(X, e, data, number_samples)

In [429]:
class Rejection_sampling: ### same as exact inference
    
    def __init__(self, X, e, data, N):
        
        self.query = X
        
        self.data = data ### possibly turn into graph/tree
        
        #print(self.data)
        
        self.query_outcomes = self.data[self.query][0]
        
        self.evidence_dict = {}
        
        e = e.split(' ')
        
        for i in range(0, len(e), 2):
            self.evidence_dict[e[i]] = e[i + 1]   
        
        self.number = N
        
        self.N = {}
        
        for i in self.data[self.query][0]:
            self.N[i] = 0
        
        #print(self.N)
            
        #print(self.evidence_dict)
        self.loop_function()
            
    
    def search_table(self, occurance, table, e): ### same as exact inference, searches for certain row
        
        indicies = []
        
        #print(e)
        
        for parent in self.data[occurance][1]:

            index = 0
            for i in self.data[parent][0]:

                if i == e[parent]:
                    indicies.append(index)
                    break
                    
                index += 1

        if len(indicies) == 1:
            return self.data[occurance][2][indicies[0]]
        
        else:
            
            fodder = table.copy()
            
            for index in indicies[:-1]:

                x = int(len(table)/len(self.data[occurance][0]))

                fodder = fodder[x*index : x*index + x]
            
            return fodder[indicies[-1]]
        
    
    def prior_sample(self): ### creates prior sample 
        
        x = {}
        for var in self.data:
            
            #creates intervals based on the outcomes of the variables 
            if self.data[var][1] == []:
                #print('here1')
                interval = self.data[var][2]
                
            else:
                #print('here2')
                interval = self.search_table(var, self.data[var][2], x) ### searches the row which will be referred to as interval
            
            #print(interval)
            
            number = random.uniform(0, 1)
            #print(number)
            
            if 0 <= number < interval[0]: ### looks for first part of the interval
                x[var] = self.data[var][0][0]
                
            elif 1 - interval[-1] <= number <= 1: ### looks in the last part of the interval
                x[var] = self.data[var][0][-1]
                
            else:
                sub_total = interval[1] ### theoretically considers more than 2 outcomes for a variables
                for i in range(1, len(interval) - 1):
                    if sub_total <= number < sub_total + interval[i + 1]:
                        x[var] = self.data[var][0][i] ### checks which interval the random number falls in 
                    sub_total += interval[i + 1]
            
            #print(x)
        return x
    
    
    def loop_function(self):
        
        for i in range(self.number): ### central engine 
            x = self.prior_sample() ### creates prior sample 
            
            if self.consistent(x):
                self.N[x[self.query]] += 1 ### adds counts to their respective values 
        
        print(self.N)
        
        print(self.normalize(self.N))
        
        #return self.N
    
    
    def normalize(self, sub_dict):
        
        total = 0
        for i in sub_dict:
            total += sub_dict[i]
            
        for i in sub_dict:
            sub_dict[i] /= total
            
        return sub_dict
    
    
    def consistent(self, e): ### checks for consistency between conditional evidence and real evidence
        
        for key in self.evidence_dict:
            if e[key] != self.evidence_dict[key]:
                return False
            
        return True
    
    

# Testing

In [496]:
xml_file = 'aima-alarm.xml'
X = 'B'
e = 'J true M true'

start = time.time()
mybninferencer(xml_file, X, e)
end = time.time()

print(end - start)
print('\n')

test_list = [1_000, 5_000, 10_000, 25_000, 50_000, 100_000]

for i in test_list:
    print(i)
    for j in range(3):
        start = time.time()
        mybn_approximate_inferencer(i, xml_file, X, e)
        end = time.time()
        print('time: ', end - start)
    print('\n')
    
for i in range(200_000, 1_000_000, 100_000):
    print(i)
    for j in range(3):  
        start = time.time()
        mybn_approximate_inferencer(i, xml_file, X, e)
        end = time.time()
        print('time: ', end - start)
    print('\n')
    
for i in range(1_000_000, 6_000_000, 1_000_000):
    print(i)
    for j in range(3):  
        start = time.time()
        mybn_approximate_inferencer(i, xml_file, X, e)
        end = time.time()
        print('time: ', end - start)
    print('\n')

{'true': 0.0005922425899999999, 'false': 0.001491857649}
{'true': 0.2841718353643929, 'false': 0.7158281646356071}
0.001995563507080078


1000
{'true': 2, 'false': 0}
{'true': 1.0, 'false': 0.0}
time:  0.008975028991699219
{'true': 0, 'false': 2}
{'true': 0.0, 'false': 1.0}
time:  0.008976221084594727
{'true': 1, 'false': 0}
{'true': 1.0, 'false': 0.0}
time:  0.008975505828857422


5000
{'true': 3, 'false': 11}
{'true': 0.21428571428571427, 'false': 0.7857142857142857}
time:  0.0359036922454834
{'true': 4, 'false': 8}
{'true': 0.3333333333333333, 'false': 0.6666666666666666}
time:  0.03503704071044922
{'true': 1, 'false': 5}
{'true': 0.16666666666666666, 'false': 0.8333333333333334}
time:  0.03390955924987793


10000
{'true': 5, 'false': 11}
{'true': 0.3125, 'false': 0.6875}
time:  0.06934642791748047
{'true': 6, 'false': 12}
{'true': 0.3333333333333333, 'false': 0.6666666666666666}
time:  0.06981205940246582
{'true': 4, 'false': 16}
{'true': 0.2, 'false': 0.8}
time:  0.069319248199462

In [497]:
xml_file = 'aima-wet-grass.xml'
X = 'R'
e = 'S true'

start = time.time()
mybninferencer(xml_file, X, e)
end = time.time()

print(end - start)
print('\n')

test_list = [1_000, 5_000, 10_000, 25_000, 50_000, 100_000]

for i in test_list:
    print(i)
    for j in range(3):
        start = time.time()
        mybn_approximate_inferencer(i, xml_file, X, e)
        end = time.time()
        print('time: ', end - start)
    print('\n')
    
for i in range(200_000, 1_000_000, 100_000):
    print(i)
    for j in range(3):  
        start = time.time()
        mybn_approximate_inferencer(i, xml_file, X, e)
        end = time.time()
        print('time: ', end - start)
    print('\n')
    
for i in range(1_000_000, 6_000_000, 1_000_000):
    print(i)
    for j in range(3):  
        start = time.time()
        mybn_approximate_inferencer(i, xml_file, X, e)
        end = time.time()
        print('time: ', end - start)
    print('\n')

{'true': 0.09000000000000001, 'false': 0.21000000000000002}
{'true': 0.3, 'false': 0.7}
0.0019953250885009766


1000
{'true': 83, 'false': 210}
{'true': 0.2832764505119454, 'false': 0.7167235494880546}
time:  0.00797581672668457
{'true': 84, 'false': 188}
{'true': 0.3088235294117647, 'false': 0.6911764705882353}
time:  0.007008552551269531
{'true': 85, 'false': 189}
{'true': 0.3102189781021898, 'false': 0.6897810218978102}
time:  0.007979154586791992


5000
{'true': 478, 'false': 1030}
{'true': 0.3169761273209549, 'false': 0.6830238726790451}
time:  0.03288388252258301
{'true': 472, 'false': 1069}
{'true': 0.3062946138870863, 'false': 0.6937053861129137}
time:  0.032917022705078125
{'true': 458, 'false': 1051}
{'true': 0.3035122597746852, 'false': 0.6964877402253148}
time:  0.03191971778869629


10000
{'true': 949, 'false': 2089}
{'true': 0.31237656352863724, 'false': 0.6876234364713627}
time:  0.059149980545043945
{'true': 848, 'false': 2164}
{'true': 0.2815405046480744, 'false': 0.71

In [498]:
xml_file = 'dog-problem.xml'
X = 'dog-out'
e = 'light-on true family-out true'

start = time.time()
mybninferencer(xml_file, X, e)
end = time.time()

print(end - start)
print('\n')

test_list = [1_000, 5_000, 10_000, 25_000, 50_000, 100_000]

for i in test_list:
    print(i)
    for j in range(3):
        start = time.time()
        mybn_approximate_inferencer(i, xml_file, X, e)
        end = time.time()
        print('time: ', end - start)
    print('\n')
    
for i in range(200_000, 1_000_000, 100_000):
    print(i)
    for j in range(3):  
        start = time.time()
        mybn_approximate_inferencer(i, xml_file, X, e)
        end = time.time()
        print('time: ', end - start)
    print('\n')
    
for i in range(1_000_000, 6_000_000, 1_000_000):
    print(i)
    for j in range(3):  
        start = time.time()
        mybn_approximate_inferencer(i, xml_file, X, e)
        end = time.time()
        print('time: ', end - start)
    print('\n')

{'true': 0.081081, 'false': 0.008918999999999998}
{'true': 0.9009, 'false': 0.09909999999999998}
0.0029926300048828125


1000
{'true': 67, 'false': 6}
{'true': 0.9178082191780822, 'false': 0.0821917808219178}
time:  0.00797891616821289
{'true': 86, 'false': 7}
{'true': 0.9247311827956989, 'false': 0.07526881720430108}
time:  0.008974790573120117
{'true': 92, 'false': 18}
{'true': 0.8363636363636363, 'false': 0.16363636363636364}
time:  0.007978439331054688


5000
{'true': 394, 'false': 36}
{'true': 0.9162790697674419, 'false': 0.08372093023255814}
time:  0.034906864166259766
{'true': 426, 'false': 43}
{'true': 0.908315565031983, 'false': 0.09168443496801706}
time:  0.034906625747680664
{'true': 418, 'false': 43}
{'true': 0.9067245119305857, 'false': 0.09327548806941431}
time:  0.034906625747680664


10000
{'true': 826, 'false': 90}
{'true': 0.9017467248908297, 'false': 0.0982532751091703}
time:  0.06881570816040039
{'true': 856, 'false': 81}
{'true': 0.9135538954108858, 'false': 0.0864

In [ ]:
class Likelihood_weighting:
    
    def __init__(self, e, bn, N):
        pass
    

In [278]:
class Gibbs:
    
    def __init__(self, e, bn, N):
        pass
    